In [90]:
import subprocess


class dualfoil:

    def __init__(self, Path='', Input='', vcutL=1e-4, vcutH=10.0):
        self.vcutL = vcutL
        self.vcutH = vcutH
        self.filePath = Path
        self.fileName = Input
        self.restart = False

        # main output list variables
        self.time = []
        self.n_util = []
        self.p_util = []
        self.potential = []
        self.uocp = []
        self.curr = []
        self.temp = []
        self.heatgen = []
        self.legs = []

    # use when wanting to start a new simulation from scratch
    def reset(self):
        self.restart = False
        self.time.clear()
        self.n_util.clear()
        self.p_util.clear()
        self.potential.clear()
        self.uocp.clear()
        self.curr.clear()
        self.temp.clear()
        self.heatgen.clear()
        self.legs.clear()

    def run(self):
        if self.filePath == '':
            subprocess.call('./dualfoil', shell=True)
        else:
            subprocess.call('cd %s && ./dualfoil' % self.filePath, shell=True)

    def set_filepath(self, path):
        if not path.endswith('/'):
            path += '/'
        self.filePath = path

    def get_voltage(self):
        if len(self.potential) != 0:
            return self.potential[-1]
        else:
            return 0.0

    def get_current(self):
        if len(self.curr) != 0:
            return self.curr[-1]
        else:
            return 0.0

    def get_total_time(self):
        rstFile = open('%sdf_restart.dat' % self.filePath, 'r')
        tmp = rstFile.readline()
        tmp = tmp.lstrip().split()
        # get timestep in minutes
        ts = float(tmp[1]) / 60
        return ts

    def get_time_step(self):
        rstFile = open('%sdf_restart.dat' % self.filePath, 'r')
        tmp = rstFile.readline()
        tmp = tmp.lstrip().split()
        # get timestep in minutes
        ts = float(tmp[0]) / 60
        return ts

    # update all list variables so that they represent last-run sim
    def update_output(self):
        t, n, p, v, u, c, tp, hg = self.extract_main_output()
        self.time += t
        self.n_util += n
        self.p_util += p
        self.potential += v
        self.uocp += u
        self.curr += c
        self.temp += tp
        self.heatgen += hg

    # append list variables to the combined output file
    def write_output(self):
        output = [self.time, self.n_util, self.p_util, self.potential,
                  self.uocp, self.curr, self.temp, self.heatgen]
        subprocess.call('date > %scombinedOutput.out' % self.filePath,
                        shell=True)

        # main output data
        with open('%scombinedOutput.out' % self.filePath, 'a') as outFile:
            outFile.write('\nMain Output data\n\n')
            outFile.write('     Time   N_util   P_util Potential   \
                          Uocp     Curr    Temp    Heatgen\n')
            outFile.write('     (min)    x         y      (v)      \
                          (v)    (A/m2)    (C)     (W/m2)\n\n')
            for i in range(len(self.time)):
                for j in range(len(output)):
                    outFile.write(str(output[j][i]).rjust(9))
                    outFile.write(',')
                    if j == (len(output)-1):
                        outFile.write('\n')

        # file for tracking simulation actions in a quickread format
        subprocess.call('date > %slegs.dat' % self.filePath, shell=True)
        with open('%slegs.dat' % self.filePath, 'a') as legsFile:
            for string in self.legs:
                string = string.rstrip('\n')
                string += '\n'  # only want one newline per command
                legsFile.write(string)

    # main computational functions----------------------------------------
    def evolve_one_time_step_constant_current(self, time_step, current,
                                              description=''):
        self.add_new_leg(description, current, time_step, 1)
        self.run()
        self.update_output()

    def evolve_one_time_step_constant_voltage(self, time_step, voltage,
                                              description=''):
        self.add_new_leg(description, voltage, time_step, 0)
        self.run()
        self.update_output()

    def evolve_one_time_step_constant_power(self, time_step, power,
                                            description=''):
        self.add_new_leg(description, power, time_step, -2)
        self.run()
        self.update_output()

    def evolve_one_time_step_constant_load(self, time_step, load,
                                           description=''):
        self.add_new_leg(description, load, time_step, -3)
        self.run()
        self.update_output()

    def evolve_to_voltage_constant_current(self, current, cutoff,
                                           description=''):
        self.add_new_leg(description, current, cutoff, 2)
        self.run()
        self.update_output()

    def evolve_one_time_step_linear_current(self, time_step, current,
                                            divisor=10, description=''):
        if not self.restart:
            tmpcurr = 0
        else:
            tmpcurr = get_current()

        ts = time_step / divisor
        ttot = ts
        change = (current-tmpcurr) / divisor
        tmpcurr = change

        while ttot <= time_step:
            self.add_new_leg(description, tmpcurr, ts, 1)
            # get next timestep values
            ttot += ts
            tmpcurr += change
            self.run()
            self.update_output()

    def evolve_one_time_step_linear_voltage(self, time_step, voltage,
                                            divisor=10, description=''):
        if not self.restart:
            tmpvolt = 0
        else:
            tmpvolt = get_current()

        ts = time_step / divisor
        tott = ts
        change = (voltage-tmpvolt) / divisor
        tmpvolt = change

        while tott <= time_step:
            self.add_new_leg(description, tmpvolt, ts, 0)
            # get next timestep values
            tott += ts
            tmpvolt += change
            self.run()
            self.update_output()

    # start_point parameter added to the following 2 functions
    # because they cannot be extracted
    def evolve_one_time_step_linear_power(self, time_step, power,
                                          start_point=0, divisor=10,
                                          description=''):
        tmppower = start_point
        ts = time_step / divisor
        tott = ts
        change = (power-start_point) / divisor
        tmppower = change

        while ts <= time_step:
            self.add_new_leg(description, tmppower, ts, 1)
            # get next timestep values
            tott += ts
            tmppower += change
            self.run()
            self.update_output()

    def evolve_one_time_step_linear_load(self, time_step, load,
                                         start_point=0, divisor=10,
                                         description=''):
        tmpload = start_point
        ts = time_step / divisor
        tott = ts
        change = (load-start_point) / divisor
        tmpload = change

        while ts <= time_step:
            self.add_new_leg(description, tmpload, ts, 1)
            # get next timestep values
            tott += ts
            tmpload += change
            self.run()
            self.update_output()

    # end main computational functions-----------------------------------------

    def add_new_leg(self, title, cu, tt, mc):

        """
        Resets input file to run the next desired simulation

        Parameters
        ----------
        title : str
            comment that describes the function of the new leg
        cu : float
            defines cu(i) for the input
            if mc = 1 or 2, cu is the current
            if mc = 0, cu is the potential
        tt : float
            defines tt(i) for the input
            if mc = 0 or 1, tt is the new leg's duration (min)
            if mc = 2, tt is the cuttof potential (V)
        mc : float
            defines mc(i) for the input, which controls mode of operation
            possible values for dualfoil 5.2 are 0, 1, and 2
        """

        newInput = ''
        modified = False
        s = self.filePath + self.fileName
        with open(s, 'r+') as file:
            line = file.readline()
            while line != '':
                # make sure restart value is set currectly
                if not self.restart:
                    if line.find('.true.') != -1:
                        line = line.replace('.true.', '.false.')
                else:
                    if line.find('.false.') != -1:
                        line = line.replace('.false.', '.true.')

                # find line before the one we need
                if line.find('lcurs') != -1 and not modified:
                    tmp = line.lstrip().split()
                    # also make sure lcurs is 1
                    if int(tmp[0]) != 1:
                        line = line.replace(str(tmp[0]), '1', 1)
                    newInput += line
                    # skip over any previous simulation command lines
                    while line != '\n':
                        line = file.readline()
                    # if next leg depends on time, we need the total time
                    if (mc == 2) or (mc == -1):
                        # depends on cutoff potential; don't alter tt(i)
                        line = (str(cu) + ' ' + str(tt) + ' ' + str(mc) + ' ' +
                                str(self.vcutL) + ' ' + str(self.vcutH) +
                                ' !' + title + '\n\n')
                    else:
                        if self.restart:
                            # depends on time AND from restart. need total
                            totT = self.get_total_time()
                            tt += totT
                        line = (str(cu) + ' ' + str(tt) + ' ' + str(mc) + ' ' +
                                str(self.vcutL) + ' ' + str(self.vcutH) +
                                ' !' + title + '\n\n')
                        self.legs.append(line)

                    # don't wanna do this ^ again within this loop
                    modified = True
                    # next leg added after this one should be from restart
                    self.restart = True

                # keep up the new file and read next line
                newInput += line
                line = file.readline()

        with open(s, 'w') as file:
            file.write(newInput)

    def extract_main_output(self):
        """
        searches for 'dualfoil5.out' file and extracts output into lists

        Returns
        -------
        time : list of float
        n_util : list of float
        p_util : list of float
        potential : list of float
        uocp : list of float
        curr : list of float
        temp : list of float
        heatgen : list of float
        """

        # first go through and find position where output starts in file
        x = 0
        previous = ''
        with open('%sdualfoil5.out' % self.filePath, 'r') as fin:
            data_list = []

            for line in fin.readlines():
                if line.find('(min)') != -1:
                    # found it! stop here
                    break
                x += 1

        # now read lines again
        with open('%sdualfoil5.out' % self.filePath, 'r') as fin:

            for line in fin.readlines()[x+2:]:
                # only take lines with convertable data
                if line.find(',') != -1:
                    # make sure we are not taking in a copy
                    if line != previous:
                        previous = line
                        line = line.rstrip('\n').rstrip(' ').lstrip(' ')
                        data_list.append(line)

        # variable lists for each time
        time = []
        n_util = []
        p_util = []
        potential = []
        uocp = []
        curr = []
        temp = []
        heatgen = []

        for data in data_list:
            tmp = data.split(',')
            for i in tmp:
                i.lstrip(' ')
            time.append(float(tmp[0]))
            n_util.append(float(tmp[1]))
            p_util.append(float(tmp[2]))
            potential.append(float(tmp[3]))
            uocp.append(float(tmp[4]))
            curr.append(float(tmp[5]))
            temp.append(float(tmp[6]))

            # for 5.1 code
            if (tmp[7] == ' ******'):
                tmp[7] = '0.00'
            heatgen.append(float(tmp[7]))

        # return data in order it appears
        return time, n_util, p_util, potential, uocp, curr, temp, heatgen


In [91]:
D = dualfoil(vcutL=0.01, vcutH=5.0)
D.reset()
D.set_filepath('/Users/ips/dualfoil')
D.evolve_one_time_step_constant_current(20.0, 20.0)
D.evolve_one_time_step_constant_voltage(2.5, 4.3)
D.evolve_one_time_step_constant_current(10.0, -10.0)
D.evolve_to_voltage_constant_current(15.0, 4.25)
D.evolve_to_voltage_constant_current(-15.0, 4.5)
print(len(D.time))
D.write_output()

1142
